In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE131528"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE131528"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE131528.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE131528.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE131528.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A composite immune signature parallels disease progression across T1D subjects"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['responder cells: UPN727'], 1: ['disease status: T1D', 'serum source - disease status: New onset T1D'], 2: ['subject: A', 'subject: B', 'subject: C', 'subject: D', 'subject: E', 'serum source - bmi: 22.609439839', 'serum source - bmi: 20.028455242', 'serum source - bmi: 16.99179819', 'serum source - bmi: 21.176533107', 'serum source - bmi: 16.617969321', 'serum source - bmi: 15.401032443', 'serum source - bmi: 20.119395077', 'serum source - bmi: 21.620647719', 'serum source - bmi: 20.436491075', 'serum source - bmi: 19.246652339', 'serum source - bmi: 20.976068397', 'serum source - bmi: 26.910153397', 'serum source - bmi: 23.269175524', 'serum source - bmi: 18.444623654', 'serum source - bmi: 20.350672

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_title and Series_summary, this seems to be a SuperSeries
# with studies on T1D subjects, likely to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Lupus/SLE): Not available directly in this dataset
# This dataset is about T1D (Type 1 Diabetes), not Lupus
# Row 1 has disease status information but it's all T1D
trait_row = None

# For age: Available in row 3 (age at enrollment) and row 4 (age at blood draw)
# Row 3 has more consistent data, so we'll use that
age_row = 3

# For gender: Available in rows 4 and 5 (serum source- sex)
# Row 5 seems to have more complete gender data
gender_row = 5

# 2.2 Data Type Conversion

# Trait conversion function (not needed since trait data is not available)
def convert_trait(value):
    return None

# Age conversion function - extract age as a continuous variable
def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract age from format like "serum source - age at enrollment: 12.1478"
    if "age at enrollment" in value:
        try:
            age_str = value.split(": ")[1]
            return float(age_str)
        except (IndexError, ValueError):
            return None
    return None

# Gender conversion function - convert to binary (female=0, male=1)
def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract gender value after colon
    if "sex" in value:
        try:
            gender = value.split(": ")[1].lower()
            if gender in ["f", "female"]:
                return 0
            elif gender in ["m", "male"]:
                return 1
        except (IndexError, ValueError):
            return None
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save metadata with initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
Found potential subseries references:
!Series_relation = SuperSeries of: GSE131523
!Series_relation = SuperSeries of: GSE131524
!Series_relation = SuperSeries of: GSE131525
!Series_relation = SuperSeries of: GSE131526
!Series_relation = SuperSeries of: GSE131527



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed, these appear to be Affymetrix probe IDs (like "1007_s_at", "1053_at", etc.)
# rather than standard human gene symbols. These identifiers will need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. First, extract the gene expression data from the matrix file
print("Extracting gene expression data from matrix file...")
# Get the genetic data from the matrix file
gene_data_probes = get_genetic_data(matrix_file)
print(f"Gene expression data has {len(gene_data_probes)} rows (probes) and {gene_data_probes.shape[1]} columns (samples)")

# 2. Get a gene mapping dataframe by extracting the probe IDs and gene symbols
print("Creating gene mapping from probe IDs to gene symbols...")
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Mapping dataframe has {len(mapping_df)} rows")
print("First few rows of the mapping dataframe:")
print(mapping_df.head())

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
print("\nApplying gene mapping to convert probe-level data to gene expression data...")
gene_data = apply_gene_mapping(expression_df=gene_data_probes, mapping_df=mapping_df)
print(f"Resulting gene expression dataframe has {len(gene_data)} rows (genes) and {gene_data.shape[1]} columns (samples)")
print("First 5 rows of gene expression data:")
print(gene_data.head())

# Save the gene expression data to file
print("\nSaving gene expression data to file...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Extracting gene expression data from matrix file...


Gene expression data has 54675 rows (probes) and 55 columns (samples)
Creating gene mapping from probe IDs to gene symbols...
Mapping dataframe has 45782 rows
First few rows of the mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Applying gene mapping to convert probe-level data to gene expression data...


Resulting gene expression dataframe has 21278 rows (genes) and 55 columns (samples)
First 5 rows of gene expression data:
          GSM3785210  GSM3785211  GSM3785212  GSM3785213  GSM3785214  \
Gene                                                                   
A1BG         5.37196     5.64512     5.45900     5.45969     5.48324   
A1BG-AS1     3.82275     3.95980     4.04396     3.88714     3.78641   
A1CF         6.13785     5.96122     6.59956     6.09580     6.61739   
A2M          5.31012     5.25374     4.74202     5.26464     5.85007   
A2M-AS1      4.92162     5.34541     5.58971     5.52085     5.38788   

          GSM3785215  GSM3785216  GSM3785217  GSM3785218  GSM3785219  ...  \
Gene                                                                  ...   
A1BG         5.35400     5.51572     5.48945     5.30483     5.53140  ...   
A1BG-AS1     3.81140     3.95647     4.02831     4.05062     3.87422  ...   
A1CF         6.43009     6.17039     5.97820     6.43666     6.43

Gene expression data saved to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE131528.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# Based on our analysis in Step 2, this dataset is about T1D, not Lupus
# Therefore, trait data is not available for our target trait (Lupus)
print("This dataset (GSE131528) contains gene expression data for T1D (Type 1 Diabetes), not Lupus.")
print("Therefore, it cannot be used for our target trait analysis.")

# For completeness, save clinical data (though it won't be linked with genetic data)
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# We're maintaining consistency with step 2 where we set trait_row = None
# This dataset doesn't contain lupus data, so we won't try to extract it
clinical_data.to_csv(out_clinical_data_file)
print(f"Saved raw clinical data to {out_clinical_data_file}")

# Create a minimal dataframe with the trait column for validation
minimal_df = pd.DataFrame({trait: []})

# Validate and save cohort information - this dataset is not usable for Lupus
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # Consistent with step 2 - no Lupus data available
    is_biased=False,  # Set to False since there's no trait data to be biased
    df=minimal_df,  # Provide a dataframe with proper structure
    note="Dataset contains T1D (Type 1 Diabetes) data, not Lupus data. Gene expression data was successfully processed but cannot be used for Lupus trait analysis."
)

print("Dataset validation complete. No linked data saved as this dataset does not contain Lupus data.")

Top 10 gene indices before normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
Top 10 gene indices after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
Shape of normalized gene data: (19845, 55)


Saved normalized gene data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE131528.csv
This dataset (GSE131528) contains gene expression data for T1D (Type 1 Diabetes), not Lupus.
Therefore, it cannot be used for our target trait analysis.
Saved raw clinical data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE131528.csv
Abnormality detected in the cohort: GSE131528. Preprocessing failed.
Dataset validation complete. No linked data saved as this dataset does not contain Lupus data.
